# TP Intelligence Artificielle - Recherche Arborescente Informée

# **Partie 0 : Visualisation d'états, Classe Taquin et Classe Node**

Nous récupérons les classes du TP précédent ainsi que la fonction visualise_state.

In [1]:
from IPython.display import display, HTML

def visualize_state(state):
    """Visualizes the given state of the Taquin using HTML."""
    html = "<table>"
    for row in state:
        html += "<tr>"
        for tile in row:
            if tile == 0:
                html += "<td style='background-color: lightgray; width: 30px; height: 30px; text-align: center; font-size: 20px;'> </td>"  # Blank tile
            else:
                html += f"<td style='background-color: lightblue; width: 30px; height: 30px; text-align: center; font-size: 20px;'>{tile}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

class Taquin:
    """
    A class representing the Taquin problem.
    """

    def __init__(self, initial_state, goal_state, size):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.size = size

    def actions(self, state):
        """Returns the possible actions (moves) from the given state."""
        # Find the position of the blank tile (0)
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Define possible moves (up, down, left, right)
        possible_actions = []
        if row > 0:
            possible_actions.append("up")
        if row < self.size-1:
            possible_actions.append("down")
        if col > 0:
            possible_actions.append("left")
        if col < self.size-1:
            possible_actions.append("right")

        return possible_actions

    def result(self, state, action):
        """Returns the state that results from applying the given action."""
        # Create a copy of the state to avoid modifying the original
        new_state = [list(row) for row in state]

        # Find the position of the blank tile (0)
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Apply the action to move the blank tile
        if action == "up":
            new_state[row][col], new_state[row - 1][col] = (
                new_state[row - 1][col],
                new_state[row][col],
            )
        elif action == "down":
            new_state[row][col], new_state[row + 1][col] = (
                new_state[row + 1][col],
                new_state[row][col],
            )
        elif action == "left":
            new_state[row][col], new_state[row][col - 1] = (
                new_state[row][col - 1],
                new_state[row][col],
            )
        elif action == "right":
            new_state[row][col], new_state[row][col + 1] = (
                new_state[row][col + 1],
                new_state[row][col],
            )

        return new_state

    def is_goal(self, state):
        return state == self.goal_state  # Directly compare with goal_state

    def cost(self, state, action):
        return 1  # Default cost is 1

class Node:
    """
    A node in a search tree.
    __init__: Initializes a node with its state, parent, action, path cost, and depth.
    __repr__: Provides a string representation of the node.
    __lt__: Defines a comparison operator for nodes based on their states.
    expand: Generates child nodes by applying all possible actions.
    child_node: Creates a single child node for a given action.
    solution: Returns the sequence of actions that led to this node.
    path: Returns the path from the root to this node as a list of nodes.
    """

    def __init__(self, state, parent=None, action=None, path_cost = 0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0 if parent is None else parent.depth + 1

    # def __repr__(self):
    #     return "<Node {}>".format(self.state)

    # def __lt__(self, other):
    #     return self.state < other.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [
            self.child_node(problem, action)
            for action in problem.actions(self.state)
        ]

    def child_node(self, problem, action):
        """Create a child node by applying the given action."""
        next_state = problem.result(self.state, action)
        next_node = Node(
            next_state,
            parent=self,
            action=action,
            path_cost = self.path_cost + problem.cost(self.state, action),
        )
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

# **Partie 1 : Heuristiques pour le jeu de Taquin**

## **1.1 H$_1$ : Nombre de tuiles mal placées**

### **Exercice 1**

Écrivez une fonction qui calcule le nombre de tuiles mal placées dans un état du jeu de Taquin

In [2]:
def misplaced_tiles(state, goal_state):
    """
    Calcule le nombre de tuiles mal placées dans un état donné.

    Args:
        state: L'état actuel du taquin.
        goal_state: L'état but du taquin.

    Returns:
        Le nombre de tuiles mal placées.
    """

    misplaced_count = 0
    for i in range(len(state)):
        for j in range(len(state[0])):
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]:  # Ignorer la tuile vide (0)
                misplaced_count += 1
    return misplaced_count

### **Exercice 2**

Calculez le nombre de tuiles mal placées pour les états suivants :
1. [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
2. [[1, 2, 3], [0, 5, 6], [4, 7, 8]]    
3. [[1, 0, 3], [4, 2, 5], [7, 8, 6]]
4. [[1, 0, 3], [4, 5, 2], [7, 6, 8]]
5. [[1, 0, 3], [4, 2, 5], [6, 7, 8]]
6. [[1, 2, 3], [4, 0, 6], [7, 5, 8]]
7. [[1, 2, 3], [0, 4, 6], [7, 5, 8]]
8. [[1, 2, 3], [4, 6, 0], [7, 5, 8]]
9. [[1, 3, 6], [4, 2, 5], [7, 0, 8]]  
10. [[1, 3, 6], [4, 2, 0], [7, 5, 8]]  
11. [[1, 3, 6], [4, 0, 2], [7, 5, 8]]  
12. [[3, 1, 2], [4, 6, 5], [7, 0, 8]]  
13. [[8, 1, 2], [0, 4, 3], [7, 6, 5]]
14. [[1, 4, 2], [7, 0, 6], [5, 3, 8]]
15. [[2, 8, 3], [1, 6, 4], [7, 0, 5]]

In [3]:
initial_states = [
    [[1, 2, 3], [4, 5, 0], [6, 7, 8]],
    [[1, 2, 3], [0, 5, 6], [4, 7, 8]],
    [[1, 0, 3], [4, 2, 5], [7, 8, 6]],
    [[1, 0, 3], [4, 5, 2], [7, 6, 8]],
    [[1, 0, 3], [4, 2, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],
    [[1, 2, 3], [0, 4, 6], [7, 5, 8]],
    [[1, 2, 3], [4, 6, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 2, 5], [7, 0, 8]],
    [[1, 3, 6], [4, 2, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 0, 2], [7, 5, 8]],
    [[3, 1, 2], [4, 6, 5], [7, 0, 8]],
    [[8, 1, 2], [0, 4, 3], [7, 6, 5]],
    [[1, 4, 2], [7, 0, 6], [5, 3, 8]],
    [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

for state in initial_states:
  print('Checking state')
  visualize_state(state)
  print('The state has',misplaced_tiles(state, goal_state),'misplaced tiles')
  print()

Checking state


1,2,3
4,5,
6,7,8


The state has 3 misplaced tiles

Checking state


1,2,3
,5,6
4,7,8


The state has 3 misplaced tiles

Checking state


...

The state has 7 misplaced tiles

Checking state


1,4,2
7,,6
5,3,8


The state has 6 misplaced tiles

Checking state


2,8,3
1,6,4
7,,5


The state has 6 misplaced tiles



## **1.2 H$_2$ : Distance de Manhattan**

### **Exercice 3**
Écrivez une fonction qui calcule la distance de Manhattan entre un état et l'état but. La fonction **next()** pourrait être utile pour trouver la position d'une tuile donnée dans l'état but.

In [5]:
def manhattan_distance(state, goal_state):
    """
    Calcule la distance de Manhattan d'un état donné à l'état but.

    Args:
        state: L'état actuel du taquin.
        goal_state: L'état but du taquin.

    Returns:
        La distance de Manhattan.
    """

    distance = 0
    for i in range(len(state)):
        for j in range(len(state[0])):
            if state[i][j] != 0:  # Ignorer la tuile vide (0)
                # Trouver la position de la tuile dans l'état but
                goal_row, goal_col = next(
                    (r, c)
                    for r, row in enumerate(goal_state)
                    for c, val in enumerate(row)
                    if val == state[i][j]
                )
                distance += abs(i - goal_row) + abs(j - goal_col)  # Distance de Manhattan
    return distance

### **Exercice 4**
1. Calculez la distance de Manhattan des états de l'exercice 2.
2. Que pouvez-vous observer par rapport aux deux heuristiques ?

In [6]:
initial_states = [
    [[1, 2, 3], [4, 5, 0], [6, 7, 8]],
    [[1, 2, 3], [0, 5, 6], [4, 7, 8]],
    [[1, 0, 3], [4, 2, 5], [7, 8, 6]],
    [[1, 0, 3], [4, 5, 2], [7, 6, 8]],
    [[1, 0, 3], [4, 2, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],
    [[1, 2, 3], [0, 4, 6], [7, 5, 8]],
    [[1, 2, 3], [4, 6, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 2, 5], [7, 0, 8]],
    [[1, 3, 6], [4, 2, 0], [7, 5, 8]],
    [[1, 3, 6], [4, 0, 2], [7, 5, 8]],
    [[3, 1, 2], [4, 6, 5], [7, 0, 8]],
    [[8, 1, 2], [0, 4, 3], [7, 6, 5]],
    [[1, 4, 2], [7, 0, 6], [5, 3, 8]],
    [[2, 8, 3], [1, 6, 4], [7, 0, 5]]
]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

for state in initial_states:
  print('Checking state')
  visualize_state(state)
  print('The state has a Manhattan distance to the goal state of',manhattan_distance(state, goal_state))
  print('The state has',misplaced_tiles(state, goal_state),'misplaced tiles')
  print()

Checking state


1,2,3
4,5,
6,7,8


The state has a Manhattan distance to the goal state of 5
The state has 3 misplaced tiles

Checking state


1,2,3
,5,6
4,7,8


The state has a Manhattan distance to the goal state of 3
The state has 3 misplaced tiles

Checking state


...

The state has a Manhattan distance to the goal state of 11
The state has 7 misplaced tiles

Checking state


1,4,2
7,,6
5,3,8


The state has a Manhattan distance to the goal state of 10
The state has 6 misplaced tiles

Checking state


2,8,3
1,6,4
7,,5


The state has a Manhattan distance to the goal state of 9
The state has 6 misplaced tiles



# **Partie 2 : Algorithme A étoile**

### **Exercice 5**

Créer une fonction A_etoile qui prend un objet problem en entrée ainsi qu'une heuristique et exécute l'algorithme A.*

Votre fonction doit :

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également un paramètre d'entrée de la fonction, avec float('inf') comme valeur par défaut.

Pour la **frontière** et l'ensemble des **nœuds déjà explorés**, vous pouvez utiliser des listes. Cependant, la frontière doit contenir les nœuds à explorer ainsi que leurs coûts (coût réel + heuristique). Pour trouver l'élément (nœud, coût) de coût minimal dans la frontière, effectuez une boucle **for** pour trouver l'indice, puis utilisez pop.

Pour ajouter un élément à une liste, vous pouvez utiliser **liste.append(element)**.

Rappelez-vous que la vérification **is_goal** doit être effectuée avant d'explorer les enfants.

In [8]:
def a_etoile(problem, heuristic, Max_it = float('inf')):
    # Initialisation de la file d'attente prioritaire (frontier) et de l'ensemble des nœuds explorés
    frontier = [(0, Node(problem.initial_state))]  # Utilisation d'une liste pour la frontier
    explored = []
    it = 0

    # Boucle principale de l'algorithme A*
    while frontier and it < Max_it:
        it = it + 1
        # Trouver le nœud ayant la valeur f la plus faible dans la frontier
        min_f = float('inf')
        min_index = -1
        for i, (f, node) in enumerate(frontier):
            if f < min_f:
                min_f = f
                min_index = i

        # Retirer le nœud ayant la valeur f la plus faible de la frontier
        _, node = frontier.pop(min_index) # We do not store the value f as we will not use it

        if problem.is_goal(node.state):
            # Solution trouvée !
            return node, it

        explored.append(node.state)

        for child in node.expand(problem):
            if child.state not in explored:
                # Calcul de la valeur f (f = g + h)
                g = child.path_cost
                h = heuristic(child.state, problem.goal_state)  # Utilisation de la fonction heuristique spécifiée
                f = g + h

                # Ajout du nœud enfant à la frontier
                frontier.append((f, child))

    # Aucune solution trouvée
    return None, it

### **Exercice 6**

Appliquez votre fonction A_etoile pour résoudre un Taquin 3x3 avec l'état initial [[1, 2, 3], [4, 5, 6], [0, 7, 8]] et les deux heuristiques.

Imprimez :
1. Le nombre de nœuds explorés par l'algorithme.
2. Le chemin permettant de passer de l'état initial à l'état objectif (la liste des actions effectuées).
3. Que pouvez-vous observer entre les deux heuristiques ?

In [12]:
initial_state = [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

print('Initial state')
visualize_state(initial_state)
problem = Taquin(initial_state, goal_state, len(initial_state[0]))
solution_node_misplaced, it_misplacec = a_etoile(problem, misplaced_tiles, Max_it = 20000)
solution_node_manhattan, it_manhattan = a_etoile(problem, manhattan_distance, Max_it = 20000)
# Print the solution (if found)
if solution_node_misplaced:
    print("Solution with 1st heuristic found in",it_misplacec,"iterations")
    print("Actions:", solution_node_misplaced.solution())
    # Visualize the solution path (optional)
    # t = 1
    # for node in solution_node_misplaced.path():
    #     print(t,'-th element of the path')
    #     visualize_state(node.state)
    #     t = t + 1
else:
    print("A* with h1 did not found a solution.")

if solution_node_manhattan:
    print("Solution with 2nd heuristic found in",it_manhattan,"iterations")
    print("Actions:", solution_node_manhattan.solution())
    # Visualize the solution path (optional)
    # t = 1
    # for node in solution_node_manhattan.path():
    #     print(t,'-th element of the path')
    #     visualize_state(node.state)
    #     t = t + 1
else:
    print("A* with h2 did not found a solution.")

Initial state


1,2,3
4,5,
6,7,8


Solution with 1st heuristic found in 196 iterations
Actions: ['down', 'left', 'left', 'up', 'right', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right']
Solution with 2nd heuristic found in 85 iterations
Actions: ['down', 'left', 'left', 'up', 'right', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right']


# **Partie 3 : Comparaison empirique de DFS, BFS, A etoile avec H1 et A etoile avec H2**

Nous allons comparer les quatre algorithmes sur les 15 instances du jeu de Taquin de l'exercice 2. Pour cela, voici les résultats de BFS et DFS sur ces 15 instances. Attention au format des tableaux. Les noms des colonnes peuvent différer de ceux que vous avez utilisés. De plus, un path_length égal à -1 signifie que l'algorithme n'a pas trouvé la solution avant d'atteindre le budget d'exploration.

In [17]:
results_BFS = [
{'initial_state': [[1, 2, 3], [4, 5, 0], [6, 7, 8]],'algorithm': 'breadth_first_search', 'num_explorations': 1846, 'path_length': 13, 'execution_time': 0.2773609161376953},
{'initial_state': [[1, 2, 3], [0, 5, 6], [4, 7, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 6, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 0, 3], [4, 2, 5], [7, 8, 6]], 'algorithm': 'breadth_first_search', 'num_explorations': 7, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 0, 3], [4, 5, 2], [7, 6, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 10911, 'path_length': 17, 'execution_time': 8.975327491760254},
{'initial_state': [[1, 0, 3], [4, 2, 5], [6, 7, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 4960, 'path_length': 15, 'execution_time': 1.831636905670166},
{'initial_state': [[1, 2, 3], [4, 0, 6], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 3, 'path_length': 2, 'execution_time': 0.0},
{'initial_state': [[1, 2, 3], [0, 4, 6], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 8, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 2, 3], [4, 6, 0], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 8, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 3, 6], [4, 2, 5], [7, 0, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 36.05873966217041},
{'initial_state': [[1, 3, 6], [4, 2, 0], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 20, 'path_length': 5, 'execution_time': 0.0},
{'initial_state': [[1, 3, 6], [4, 0, 2], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 62, 'path_length': 6, 'execution_time': 0.0019948482513427734},
{'initial_state': [[3, 1, 2], [4, 6, 5], [7, 0, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 40.92391228675842},
{'initial_state': [[8, 1, 2], [0, 4, 3], [7, 6, 5]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 40.67150044441223},
{'initial_state': [[1, 4, 2], [7, 0, 6], [5, 3, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 3321, 'path_length': 14, 'execution_time': 0.8201050758361816},
{'initial_state': [[2, 8, 3], [1, 6, 4], [7, 0, 5]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 42.47766828536987}
]
results_DFS = [
{'initial_state': [[1, 2, 3], [4, 5, 0], [6, 7, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 69.51113247871399},
{'initial_state': [[1, 2, 3], [0, 5, 6], [4, 7, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 27, 'path_length': 27, 'execution_time': 0.0},
{'initial_state': [[1, 0, 3], [4, 2, 5], [7, 8, 6]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 60.40420460700989},
{'initial_state': [[1, 0, 3], [4, 5, 2], [7, 6, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 62.47419261932373},
{'initial_state': [[1, 0, 3], [4, 2, 5], [6, 7, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 67.59605073928833},
{'initial_state': [[1, 2, 3], [4, 0, 6], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 412, 'path_length': 406, 'execution_time': 0.02211737632751465},
{'initial_state': [[1, 2, 3], [0, 4, 6], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 65.1717848777771},
{'initial_state': [[1, 2, 3], [4, 6, 0], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 886, 'path_length': 871, 'execution_time': 0.08270859718322754},
{'initial_state': [[1, 3, 6], [4, 2, 5], [7, 0, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 66.5749568939209},
{'initial_state': [[1, 3, 6], [4, 2, 0], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 65.61762142181396},
{'initial_state': [[1, 3, 6], [4, 0, 2], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 66.86844420433044},
{'initial_state': [[3, 1, 2], [4, 6, 5], [7, 0, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 80.54781603813171},
{'initial_state': [[8, 1, 2], [0, 4, 3], [7, 6, 5]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 75.41818499565125},
{'initial_state': [[1, 4, 2], [7, 0, 6], [5, 3, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 69.51109766960144},
{'initial_state': [[2, 8, 3], [1, 6, 4], [7, 0, 5]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 83.47795271873474}
]

### **Exercice 7**
Utilisez (et adaptez si nécessaire) votre fonction compare_algorithms du premier TP pour résoudre tous les jeux avec les deux versions de l'algorithme A etoile.

In [14]:
import time

def compare_algorithms(initial_states, goal_state, heuristics, max_it):
    results = [[] for _ in range(len(heuristics))]
    for initial_state in initial_states:
        print('Solving state')
        visualize_state(initial_state)
        problem = Taquin(initial_state, goal_state, len(initial_state[0]))
        for k in range(len(heuristics)):
            heuristic = heuristics[k]
            print('Running A* with', heuristic.__name__)
            start_time = time.time()
            solution_node, num_explorations = a_etoile(problem,heuristic,max_it)
            end_time = time.time()
            execution_time = end_time - start_time

            if solution_node:
                path_length = len(solution_node.solution())
            else:
                path_length = -1  # Indicate no solution found

            results[k].append({
                "initial_state": initial_state,
                "algorithm": heuristic.__name__,  # Get the name of the function
                "num_explorations": num_explorations,
                "path_length": path_length,
                "execution_time": execution_time
            })
    return results

In [15]:
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
heuristics = [misplaced_tiles, manhattan_distance]

results = compare_algorithms(initial_states, goal_state, heuristics,max_it = 20000)
results_misplace = results[0]
results_manhattan = results[1]
print('Finished !!')

Solving state


1,2,3
4,5,
6,7,8


Running A* with misplaced_tiles
Running A* with manhattan_distance
Solving state


1,2,3
,5,6
4,7,8


Running A* with misplaced_tiles
Running A* with manhattan_distance
Solving state


...

Running A* with misplaced_tiles
Running A* with manhattan_distance
Solving state


1,4,2
7,,6
5,3,8


Running A* with misplaced_tiles
Running A* with manhattan_distance
Solving state


2,8,3
1,6,4
7,,5


Running A* with misplaced_tiles
Running A* with manhattan_distance


#### **Utilisez le code ci-dessous pour visualiser vos résultats et commentez**

In [18]:
import pandas as pd
import numpy as np
from tabulate import tabulate ## La librairie Tabulate doit être installée. Si vous ne l'avez pas, vous pouvez simplement utiliser "print(df)" pour visualiser le tableau.

Data = []
for i in range(len(initial_states)):
    Data.append([int(i+1),
                 results_BFS[i]['num_explorations'], results_DFS[i]['num_explorations'], results_misplace[i]['num_explorations'], results_manhattan[i]['num_explorations'],
                 results_BFS[i]['path_length'], results_DFS[i]['path_length'], results_misplace[i]['path_length'], results_manhattan[i]['path_length'],
                 round(results_BFS[i]['execution_time'],2), round(results_DFS[i]['execution_time'],2), round(results_misplace[i]['execution_time'],2), round(results_manhattan[i]['execution_time'],2)])

df = pd.DataFrame(Data, columns=["State", "Nodes explored BFS", "Nodes explored DFS", "Nodes explored A*H1", "Nodes explored A*H2",
                                 "Path length BFS", "Path length DFS", "Path length A*H1", "Path length A*H2",
                                 "Execution Time BFS", "Execution Time DFS","Execution Time A*H1", "Execution Time A*H2"])
print(tabulate(df, headers='keys', tablefmt='pretty'))

+----+-------+--------------------+--------------------+---------------------+---------------------+-----------------+-----------------+------------------+------------------+--------------------+--------------------+---------------------+---------------------+
|    | State | Nodes explored BFS | Nodes explored DFS | Nodes explored A*H1 | Nodes explored A*H2 | Path length BFS | Path length DFS | Path length A*H1 | Path length A*H2 | Execution Time BFS | Execution Time DFS | Execution Time A*H1 | Execution Time A*H2 |
+----+-------+--------------------+--------------------+---------------------+---------------------+-----------------+-----------------+------------------+------------------+--------------------+--------------------+---------------------+---------------------+
| 0  |  1.0  |       1846.0       |      20000.0       |        196.0        |        85.0         |      13.0       |      -1.0       |       13.0       |       13.0       |        0.28        |       69.51        | 